In [1]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from markov import *
import utils

In [2]:
X0,X1=utils.cgi_partition('AL031718.fas', [(4620, 8266), (16033, 18297)])

X=np.array(X0+X1)
Y=np.array([0]*len(X0)+[1]*len(X1)) # labels (0=non CpG, 1=CpG)

In [16]:
# states=['A','C','G','T']
transitions=[['AA','AC','AG','AT'],
             ['CA','CC','CG','CT'],
             ['GA','GC','GG','GT'],
             ['TA','TC','TG','TT'],]

M=Markov()

# k-fold cross validation
k=10
cerror=crecall=cprecision=0
skf=StratifiedKFold(n_splits=k, shuffle=True)
for train, test in skf.split(X, Y):
  X_train, X_test=X[train], X[test]
  Y_train, Y_test=Y[train], Y[test]

  X0,X1=X_train[Y_train==0],X_train[Y_train==1]
  M.train(X0, X1, transitions)
  Y_pred=M.predict_all(X_test)

  tp=np.sum((Y_pred==1)&(Y_test==1))
  tn=np.sum((Y_pred==0)&(Y_test==0))
  fp=np.sum((Y_pred==1)&(Y_test==0))
  fn=np.sum((Y_pred==0)&(Y_test==1))

  cerror+=(fp+fn)/len(Y_test)
  cprecision=tp/(tp+fp) if (tp+fp)>0 else 0
  crecall=tp/(tp+fn) if (tp+fn)>0 else 0

cerror/=k
cprecision/=k
crecall/=k
f1=(2*cprecision*crecall)/(cprecision+crecall) if (cprecision+crecall)>0 else 0

print(f'Error rate = {cerror}\n')
# print(f'Precision = {cprecision}\nRecall = {crecall}\nF1 = {f1}')

Error rate = 0.11043956043956045

